# Сведение данных по дню, метрик и остатков, YANDEX заказы

### Импорты

In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import os
import glob
from sys import exit
import sys
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm

#### Константы для расчета наценки

In [2]:
proc_priem_plat = 1.6 
proc_progr_lo = 0.7
proc_sklad_obr = 3 
proc_posl_mil = 5.5 

posl_mil_porog_min = 13 
posl_mil_porog_max = 350 
sklad_obr_porog_min = 20 
sklad_obr_porog_max = 60 


#### Каталог товаров

In [3]:
## Товары - Каталог - Файл с товарами
path_ya_catalog = path_downloads + f"mass_assortment_business_1075309_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"

#### ФБС остатки

In [4]:
## Товары - Ассортимент по магазинам - Нужный фбс магазин - Скачать - Файл с товарами
path_fbs_stocks = [
                   path_downloads + f"mass_assortment_69805661_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx",
                   path_downloads + f"mass_assortment_71935156_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx",
#                    path_downloads + f"mass_assortment_76326275_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"
                   ]

#### Цены

In [5]:
## Товары - Цены - аккаунт Интернет Революция - Цены и автоматические стратегии
path_price = path_downloads + f"mass_assortment_price_1075309_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"

#### Дополнительные цены для отчета по ценам

In [6]:
## Товары – Цены – Цены на рынке – Столбец «Ваша цена, ₽» для каждого кабинета
path_price_for_PRICES = path_downloads + f"supplier_prices_v2_1075309_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"
path_price_for_PRICES_himki = path_downloads + f"supplier_prices_v2_1075309_{mwm.dt_now.date().strftime('%d-%m-%Y')} (1).xlsx"
path_price_for_PRICES_irkutsk = path_downloads + f"supplier_prices_v2_1075309_{mwm.dt_now.date().strftime('%d-%m-%Y')} (2).xlsx"

#### Остатки

In [7]:
## Логистика - Остатки на складах - Сформировать - Скачать (в аккаунте Интернет Ревлюция)
path_file2 = path_downloads + f"stocks_on_warehouses_22081242_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"

#### Заказы

In [8]:
## Бухгалтерия - Финансовые отчеты - По заказам - Нужный день - Скачать (в аккаунте Интернет Ревлюция)
if mwm.dt_now.date().isoweekday() != 1:
    P = [path_downloads + f"united_orders_22081242_{(mwm.dt_now.date() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')}_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"]
else:
    P = [
         path_downloads +  f"united_orders_22081242_{(mwm.dt_now.date() - datetime.timedelta(days=3)).strftime('%d-%m-%Y')}_{(mwm.dt_now.date() - datetime.timedelta(days=2)).strftime('%d-%m-%Y')}.xlsx",
         path_downloads +  f"united_orders_22081242_{(mwm.dt_now.date() - datetime.timedelta(days=2)).strftime('%d-%m-%Y')}_{(mwm.dt_now.date() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')}.xlsx",
         path_downloads +  f"united_orders_22081242_{(mwm.dt_now.date() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')}_{mwm.dt_now.date().strftime('%d-%m-%Y')}.xlsx"
        ]

#### Акции

In [9]:
## файл с акциями(акцией) на яндексе; Продвижение - Акции - Нужная акция - Загрузить файл - Выгрузить список - Скачать
# Уже идут
path_AC = [
'result_42018bec-7b63-4eea-948c-2e897c73e873-1712633981.xlsx', #Маркет брендов
'result_98792915-5cda-4010-8f5a-7510d84a1cf6-1712633998.xlsx', #Скидки для FBY
]

# #Будут идти
path_AC_future = [
'result_307bf71c-d18c-401a-b66a-8429210964e5-1711510500.xlsx', #Флеш: космические товары. 11.04–12.04
]


#### Файл от Алены (для процента размещения на витрине)

In [10]:
p_p = path_downloads + 'Яндекс цены, наценка 1.10.23.xlsx'

In [11]:
nac_flag = False
reestr = wwm.Reestr()

def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col 

def letter_substitution(data):
    
    data2 = data.copy()
    
    data2 = data2.rename(columns={data2.columns[0]: 'Артикул'})
    data2['Артикул'] = data2['Артикул'].astype(str)
    data2 = data2.reset_index(drop=True)
    
    fa = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data2['Артикул'])))

    if len(fa) > 0:

        Q = {'е': 'e', 
             'Е': 'E', 
             'Т': 'T', 
             'у': 'y', 
             'о': 'o', 
             'О': 'O', 
             'р': 'p', 
             'Р': 'P', 
             'а': 'a', 
             'А': 'A', 
             'Н': 'H',
             'К': 'K',
             'х': 'x',
             'Х': 'X',
             'с': 'c',
             'С': 'C',
             'В': 'B',
             'М': 'M'}

        for i in range(len(data2)):

            for row in list(Q.keys()):

                if row in data2.loc[i]['Артикул']:

                    data2['Артикул'].loc[i] = data2.loc[i]['Артикул'].replace(row, Q[row])
                    
                    
                    
                    
        fa2 = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data2['Артикул'])))

        if len(fa2) == 0:

            return data2

    else:
        
        return data

In [12]:
p = pd.read_excel(p_p, sheet_name='с 1.10')
#p = pd.read_excel(p_p, sheet_name='с 1.09')

p = p[['Артикул', 'размещение на витрине %']]
p['Артикул'] = p['Артикул'].astype(str)
p = letter_substitution(p)
p = p.drop_duplicates().reset_index(drop=True)

if p['размещение на витрине %'].mean() < 1:    
    p['размещение на витрине %'] = p['размещение на витрине %'] * 100

    
print(len(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(p['Артикул'])))))
print(len(p)-len(list(set(p['Артикул']))))

0
0


In [13]:
kat = pd.read_excel(path_ya_catalog,
                   sheet_name='Список товаров',
                   skiprows=1,
                   usecols=['Ваш SKU *', 
                            'Категория в вашем магазине *', 
                            'Вес с упаковкой, кг', 
                            'Габариты с упаковкой, см'])\
      .rename(columns={'Ваш SKU *': 'Артикул'})\
      .iloc[1:, :].reset_index(drop=True)

In [14]:
kat['Артикул'] = kat['Артикул'].astype(str)

In [15]:
kat = kat.query("Артикул not in ['А1301', 'А1401', 'А1601', 'А1801', 'А2501', 'А4201', 'А5801']").reset_index(drop=True)

In [16]:
len(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(kat['Артикул']))))

53

In [17]:
kat = letter_substitution(kat)

In [18]:
len(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(kat['Артикул']))))

0

In [19]:
len(kat) - len(list(set(kat['Артикул'])))

5

In [20]:
kat = kat.drop_duplicates().reset_index(drop=True)

In [21]:
Q = []
for i in range(len(kat)):
    
    art = kat.loc[i]['Артикул']
    
    if len(kat[kat['Артикул']==art]) > 1:
        display(kat[kat['Артикул']==art])
        
        Q.append(i)

,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
512,A2102,Приспособления для яиц,0.11,6/2/10
1108,A2102,Аксессуары для готовки,0.11,6/10/2


,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
567,A6701,Бутылка,0.485,7/22/7
1136,A6701,Бутылки для напитков,0.485,22/6.5/0.7


,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
580,A8601,"Наборы и емкости для хранения специй и соусов,...",0.385,18/4.5/4.5
1152,A8601,"Солонки, перечницы и емкости для специй",0.385,18/4.5/4.5


,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
512,A2102,Приспособления для яиц,0.11,6/2/10
1108,A2102,Аксессуары для готовки,0.11,6/10/2


,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
567,A6701,Бутылка,0.485,7/22/7
1136,A6701,Бутылки для напитков,0.485,22/6.5/0.7


,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см"
580,A8601,"Наборы и емкости для хранения специй и соусов,...",0.385,18/4.5/4.5
1152,A8601,"Солонки, перечницы и емкости для специй",0.385,18/4.5/4.5


In [22]:
Q = list(set(Q))

In [23]:
# kat.query('index in @Q').sort_values(by='Артикул').reset_index(drop=True).to_excel(path_downloads + 'Дубли в категориях яндекс.xlsx', index=False)


In [24]:
Q = []
for i in range(len(kat)):
    Q.append(round(eval(str([float(x) for x in kat.loc[i]['Габариты с упаковкой, см'].split('/')])[1:-1].replace(',', '*')) / 5000, 3))
    
kat['Габариты'] = Q

In [25]:
kat[['Габариты', 'Вес с упаковкой, кг']] = kat[['Габариты', 'Вес с упаковкой, кг']].astype(float)

In [26]:
Q = []
for i in range(len(kat)):
    if kat.loc[i]['Вес с упаковкой, кг'] > kat.loc[i]['Габариты']:
        Q.append(kat.loc[i]['Вес с упаковкой, кг'])
    else:
        Q.append(kat.loc[i]['Габариты'])

kat['Объемный вес, кг'] = Q

In [27]:
kat

,Артикул,Категория в вашем магазине *,"Вес с упаковкой, кг","Габариты с упаковкой, см",Габариты,"Объемный вес, кг"
0,16801,Чехлы для планшетов,0.265,24.5/18/18,1.588,1.588
1,16802,Чехлы для планшетов,0.265,24.5/18/18,1.588,1.588
2,16803,Чехлы для планшетов,0.265,24.5/18/18,1.588,1.588
3,16804,Чехлы для планшетов,0.265,24.5/18/18,1.588,1.588
4,16920,Нож канцелярский,0.120,14/2/1,0.006,0.120
...,...,...,...,...,...,...
1148,A8202,Перчатки хозяйственные,0.165,3/15.5/16.5,0.153,0.165
1149,A8203,Перчатки хозяйственные,0.165,3/15.5/16.5,0.153,0.165
1150,A8301,"Солонки, перечницы и емкости для специй",0.950,18.5/18.5/25.7,1.759,1.759
1151,A8501,Скалки,0.100,53/4/4,0.170,0.170


In [28]:
len(kat) - len(list(set(kat['Артикул'])))

3

In [29]:
obvy = kat[['Артикул', 'Объемный вес, кг']].copy()
obvy = obvy.drop_duplicates().reset_index(drop=True)

In [30]:
obvy

,Артикул,"Объемный вес, кг"
0,16801,1.588
1,16802,1.588
2,16803,1.588
3,16804,1.588
4,16920,0.120
...,...,...
1145,A8201,0.165
1146,A8202,0.165
1147,A8203,0.165
1148,A8301,1.759


In [31]:
len(obvy) - len(list(set(obvy['Артикул'])))

0

In [32]:
# файл с метриками по остаткам

path_stocks_ya_metrics = path_algoritm + 'YANDEX_ORDERS/stocks_ya_metrics.xlsx'


# файл с метриками по ценам

path_ya_price_metrics = path_algoritm + 'YANDEX_ORDERS/ya_price_metrics.xlsx'


# файл с метриками по остаткам fbs

path_stocks_fbs_ya_metrics = path_algoritm + 'YANDEX_ORDERS/stocks_fbs_ya_metrics.xlsx'

In [33]:
# файл со Статусами

path_st = wwm.path_st
st = pd.read_excel(path_st, dtype={'Артикул': 'str'})

In [34]:
## файл с метриками общими, YANDEX заказы (orders_metrics!)

path_file3 = path_algoritm + 'YANDEX_ORDERS/yandex_orders_metrics!.csv'

In [35]:
# Обновление таблиц по яндексу

ya_catalog = pd.read_excel(path_ya_catalog,     
                           sheet_name='Список товаров',
                           skiprows = 2,
                           usecols=[2, 3, 8],
                           names=['Артикул', 'Название', 'Номенклатура'],
                           dtype={'Артикул':'str'})
ya_catalog = letter_substitution(ya_catalog)
ya_catalog = ya_catalog.drop_duplicates().reset_index(drop=True)
Q = []
for i in range(len(ya_catalog)):
    art = ya_catalog.loc[i]['Артикул']
    if len(ya_catalog[ya_catalog['Артикул']==art]) > 1:
        for row in list(ya_catalog[ya_catalog['Артикул']==art]['Номенклатура'].values):
            if ',' in row:
                for ind in list(ya_catalog[ya_catalog['Артикул']==art].index):
                    if ',' not in ya_catalog.loc[ind]['Номенклатура']:
                        Q.append(ind)
Q = list(set(Q))
ya_catalog = ya_catalog.query('index not in @Q').reset_index(drop=True)
Q = []
for i in range(len(ya_catalog)):
    art = ya_catalog.loc[i]['Артикул']
    if len(ya_catalog[ya_catalog['Артикул']==art]) > 1:
        Q.append(list(ya_catalog[ya_catalog['Артикул']==art].index)[-1])
ya_catalog = ya_catalog.query('index not in @Q').reset_index(drop=True)
if len(ya_catalog) - len(list(set(ya_catalog['Артикул']))) !=0:
    print('error')
    int('error')
else:
    None
    
    
stocks_ya_metrics = pd.read_excel(path_stocks_ya_metrics, dtype={'Артикул':'str'})
stocks_ya_metrics = stocks_ya_metrics.merge(ya_catalog[['Артикул']], on='Артикул', how='outer')

if len(stocks_ya_metrics) - len(list(set(stocks_ya_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    stocks_ya_metrics.to_excel(path_stocks_ya_metrics, index=False)
    

    
stocks_fbs_ya_metrics = pd.read_excel(path_stocks_fbs_ya_metrics, dtype={'Артикул':'str'})
stocks_fbs_ya_metrics = stocks_fbs_ya_metrics.merge(ya_catalog[['Артикул']], on='Артикул', how='outer')

if len(stocks_fbs_ya_metrics) - len(list(set(stocks_fbs_ya_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    stocks_fbs_ya_metrics.to_excel(path_stocks_fbs_ya_metrics, index=False)
    
    
    
ya_price_metrics = pd.read_excel(path_ya_price_metrics, dtype={'Артикул':'str'})
ya_price_metrics = ya_price_metrics.merge(ya_catalog[['Артикул']], on='Артикул', how='outer')

if len(ya_price_metrics) - len(list(set(ya_price_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    ya_price_metrics.to_excel(path_ya_price_metrics, index=False)
    
    
    
metrics = pd.read_csv(path_file3, sep=';')
metrics = metrics.merge(ya_catalog, on='Артикул', how='outer')

metrics['Название_x'] = metrics['Название_x'].fillna(metrics['Название_y'])
metrics['Номенклатура_x'] = metrics['Номенклатура_x'].fillna(metrics['Номенклатура_y'])
metrics = metrics.drop(['Название_y', 'Номенклатура_y'], axis=1)
metrics = metrics.rename(columns={'Название_x': 'Название', 'Номенклатура_x': 'Номенклатура'})


metrics = metrics.merge(
                        letter_substitution(
                                            pd.read_excel(path_file2,
                                                          skiprows=4, 
                                                          usecols=['Unnamed: 0', 'Unnamed: 1'],
                                                          dtype={'Unnamed: 0': 'str',
                                                                 'Unnamed: 1': 'int'})\
                                            .rename(columns={'Unnamed: 0': 'Артикул', 
                                                             'Unnamed: 1': 'Номенклатура_доп.'}))\
                        .drop_duplicates(subset='Артикул', keep='first').reset_index(drop=True),
                        on='Артикул',
                        how='left')

metrics['Номенклатура'] = metrics['Номенклатура_доп.']
metrics = metrics.drop('Номенклатура_доп.', axis=1)


metrics = metrics.merge(
                        letter_substitution(
                                            pd.read_excel(path_ya_catalog, 
                                                          sheet_name='Список товаров', 
                                                          skiprows=1, 
                                                          usecols=['Ваш SKU *', 'SKU на Маркете'],
                                                          dtype={'Ваш SKU *': 'str'}).iloc[1:, :]\
                                            .rename(columns={'Ваш SKU *': 'Артикул', 
                                                             'SKU на Маркете': 'Номенклатура_доп.'}))\
                        .drop_duplicates(subset='Артикул', keep='first').reset_index(drop=True),
                        on='Артикул',
                        how='left')

metrics['Номенклатура'] = metrics['Номенклатура_доп.']
metrics = metrics.drop('Номенклатура_доп.', axis=1)



if len(metrics) - len(list(set(metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    metrics.to_csv(path_file3, index=False, sep=";", encoding='utf-8-sig')
    
stocks_ya_metrics = None
stocks_fbs_ya_metrics = None
ya_price_metrics = None
metrics = None

## Преобразование fbs остатков и добавление в метрики по остаткам

In [36]:
stocks_fbs_ya_metrics = pd.read_excel(path_stocks_fbs_ya_metrics, dtype={'Артикул': 'str'})

if len(stocks_fbs_ya_metrics) - len(list(set(stocks_fbs_ya_metrics['Артикул']))) != 0:
    print('error')
    int('error')
else:
    None
    
stocks_fbs_ya_metrics = letter_substitution(stocks_fbs_ya_metrics)  

date_for_fbs_stocks_list = []
for path in path_fbs_stocks:
    if '69805661' in path.split('assortment')[1] or '76326275' in path.split('assortment')[1] \
    or '71935156' in path.split('assortment')[1]\
    or '1075309' in path.split('assortment')[1]:
        date_for_fbs_stocks = pd.to_datetime(path\
                                             .split('assortment')[1]\
                                             .split('.')[0][-10:], format='%d-%m-%Y')\
                                             .strftime('%d-%B-%Y')
        
        fbs_stocks = pd.read_excel(path, 
                                   sheet_name='Ассортимент', 
                                   skiprows=1, 
                                   usecols=['Ваш SKU *', 'Остаток'],
                                   dtype={'Ваш SKU *': 'str'}).iloc[1:, :]
    
        fbs_stocks.columns = ['Артикул', 'Остаток fbs']
        fbs_stocks = fbs_stocks.dropna().reset_index(drop=True)
        fbs_stocks['Остаток fbs'] = fbs_stocks['Остаток fbs'].astype(int)
        fbs_stocks = letter_substitution(fbs_stocks)
        fbs_stocks = fbs_stocks.drop_duplicates().reset_index(drop=True)
        Q = []
        for i in range(len(fbs_stocks)):
            art = fbs_stocks.loc[i]['Артикул']
            dat = fbs_stocks[fbs_stocks['Артикул']==art]
            if len(dat) > 1:
                for ind in list(dat.index):
                    if dat.loc[ind]['Остаток fbs'] != sorted(dat['Остаток fbs'].values)[-1]:
                        Q.append(ind)
        Q = list(set(Q))
        fbs_stocks = fbs_stocks.query('index not in @Q').reset_index(drop=True) 

        if len(fbs_stocks) - len(list(set(fbs_stocks['Артикул']))) != 0:
            print('error')
            int('error')
        else:
            None
        
        date_for_fbs_stocks_list.append(date_for_fbs_stocks)
            
        if path == path_fbs_stocks[0]:
            
            fbs_stocks_0 = fbs_stocks.copy()
        
        else:
            
            fbs_stocks = pd.concat([fbs_stocks_0, fbs_stocks], ignore_index=True)\
                         .groupby('Артикул')['Остаток fbs'].sum()\
                         .reset_index(drop=False)
    else:
        print('Не тот файл')
        int('error')
            
if len(list(set(date_for_fbs_stocks_list))) != 1:
    print('error')
    exit()

date_for_fbs_stocks = date_for_fbs_stocks_list[0]

if date_for_fbs_stocks not in stocks_fbs_ya_metrics.columns:
    fbs_stocks_for_metrics = fbs_stocks.copy()
    fbs_stocks_for_metrics.columns = ['Артикул', date_for_fbs_stocks]
    stocks_fbs_ya_metrics = stocks_fbs_ya_metrics.merge(fbs_stocks_for_metrics, on='Артикул', how='outer')
    if len(stocks_fbs_ya_metrics) - len(list(set(stocks_fbs_ya_metrics['Артикул']))) != 0:
        print('error')
        int('error')
    else:

        stocks_fbs_ya_metrics = stocks_fbs_ya_metrics[list(stocks_fbs_ya_metrics.columns[:1])\
                                + [y.strftime('%d-%B-%Y') for y in\
                                sorted([pd.to_datetime(x) for x in dt_col(stocks_fbs_ya_metrics)])]]

        stocks_fbs_ya_metrics.to_excel(path_stocks_fbs_ya_metrics, index=False)
        fbs_stocks_for_metrics = None
else:
    None
    


In [37]:
# файл с себестоимостью (общий)

path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'

In [38]:
# файл с признаком "В резерве" (общий)

path_vr = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/В резерве/В резерве_обработанные.xlsx'


In [39]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col

In [40]:
def Apocalypse(n=-1):

    stocks_ya_metrics = pd.read_excel(path_stocks_ya_metrics)
    stocks_ya_metrics = stocks_ya_metrics.iloc[:, :n]
    stocks_ya_metrics.to_excel(path_stocks_ya_metrics, index=False)
    
    stocks_fbs_ya_metrics = pd.read_excel(path_stocks_fbs_ya_metrics)
    stocks_fbs_ya_metrics = stocks_fbs_ya_metrics.iloc[:, :-1]
    stocks_fbs_ya_metrics.to_excel(path_stocks_fbs_ya_metrics, index=False)

    ya_price_metrics = pd.read_excel(path_ya_price_metrics)
    ya_price_metrics = ya_price_metrics.iloc[:, :n]
    ya_price_metrics.to_excel(path_ya_price_metrics, index=False)

    metrics = pd.read_csv(path_file3, sep=';')
    metrics = metrics[list(metrics.columns[:3])+dt_col(metrics)[:n]+list(metrics.columns[-2:])]
    metrics.to_csv(path_file3, index=False, sep=";", encoding='utf-8-sig')

In [41]:
# Apocalypse()

In [42]:
# Apocalypse(-3)

In [43]:
ym = pd.read_excel(path_downloads + 'яндекс.xlsx')
ym['Артикул'] = ym['Артикул'].astype(str)
ym = ym[['Артикул', 'новинка']]

In [44]:
st = pd.read_excel(path_st)
st['Артикул'] = st['Артикул'].astype(str)

In [45]:
print(len(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(st['Артикул'])))))
print(len(st)-len(list(set(st['Артикул']))))


0
0


In [46]:
path_price

'/Users/dmitriymaslov/Мой диск/Загрузки Маслов/mass_assortment_price_1075309_09-04-2024.xlsx'

In [47]:
price = pd.read_excel(path_price, sheet_name='Цены')
price.columns = list(price.loc[0])
price = price.iloc[2:, :].reset_index(drop=True)
price = price[['Ваш SKU *', 'Базовая цена', 'Базовая цена до скидки']]
price.columns = ['Артикул', 'Цена', 'Цена до скидки']
price = letter_substitution(price)
price['Артикул'] = price['Артикул'].astype(str)
dop_price = price[['Артикул', 'Цена до скидки']].copy()
price = price[['Артикул', 'Цена']]
price = price.dropna().reset_index(drop=True)
dop_price = dop_price.dropna().reset_index(drop=True)
price = price.drop_duplicates().reset_index(drop=True)
dop_price = dop_price.drop_duplicates().reset_index(drop=True)

In [48]:
Q = []
for i in range(len(price)):
    art = price.loc[i]['Артикул']
    if len(price[price['Артикул']==art]) > 1:
        price_art_list = sorted(list(price[price['Артикул']==art]['Цена'].values))[1:]
        Q += list(price[(price['Цена']\
             .isin(price_art_list))&(price['Артикул']==art)].index)
Q = list(set(Q))
price = price.query('index not in @Q').reset_index(drop=True)


In [49]:
Q = []
for i in range(len(dop_price)):
    art = dop_price.loc[i]['Артикул']
    if len(dop_price[dop_price['Артикул']==art]) > 1:
        price_art_list = sorted(list(dop_price[dop_price['Артикул']==art]['Цена до скидки'].values))[1:]
        Q += list(dop_price[(dop_price['Цена до скидки']\
             .isin(price_art_list))&(dop_price['Артикул']==art)].index)
Q = list(set(Q))
dop_price = dop_price.query('index not in @Q').reset_index(drop=True)

In [50]:
if len(price) - len(list(set(price[price.columns[0]]))) == 0:
    None
else:
    price = None

In [51]:
price[price['Артикул']=='А2502']

,Артикул,Цена


In [52]:
if len(dop_price) - len(list(set(dop_price[dop_price.columns[0]]))) == 0:
    None
else:
    dop_price = None

In [53]:
dop_price

,Артикул,Цена до скидки
0,16801,1071
1,16802,999
2,16803,1191
3,16804,992
4,16920,429
...,...,...
1143,A8202,733
1144,A8203,609
1145,A8301,3290
1146,A8501,1003


## Загрузка таблицы с метриками

In [54]:
path_file3

'/Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/YANDEX_ORDERS/yandex_orders_metrics!.csv'

In [55]:
yandex_orders_metrics  = pd.read_csv(path_file3, sep=';')

In [56]:
yandex_orders_metrics 

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,Остаток,Цена
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,722
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,601
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,48,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,501
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,18,440
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590


In [57]:
# yandex_orders_metrics = yandex_orders_metrics.drop_duplicates().reset_index(drop=True)

In [58]:
yandex_orders_metrics['Номенклатура'].value_counts()

1.019762e+11    1
1.016993e+11    1
1.016993e+11    1
1.016993e+11    1
1.016993e+11    1
               ..
1.017030e+11    1
1.018271e+11    1
1.019121e+11    1
1.018523e+11    1
1.029119e+11    1
Name: Номенклатура, Length: 1150, dtype: int64

In [59]:
yandex_orders_metrics['Артикул'] = yandex_orders_metrics['Артикул'].astype(str)
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [60]:
for i in range(len(yandex_orders_metrics)):
    
    if len(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']]) > 1:
        
        display(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']])

In [61]:
if len(yandex_orders_metrics[yandex_orders_metrics['Артикул']=='A2102']) > 1:
    if 250 in list(yandex_orders_metrics[yandex_orders_metrics['Артикул']=='A2102']['Цена'].values):
         yandex_orders_metrics = yandex_orders_metrics.query(f"index not in {list(yandex_orders_metrics[(yandex_orders_metrics['Артикул']=='A2102')&(yandex_orders_metrics['Цена']==250)].index)}").reset_index(drop=True)



In [62]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [63]:
for i in range(len(yandex_orders_metrics)):
    
    if len(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']]) > 1:
        
        display(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']])

In [64]:
Q = []
for i in range(len(yandex_orders_metrics)):
    
    if yandex_orders_metrics.loc[i]['Артикул'] == 'A7201' and yandex_orders_metrics.loc[i]['Цена'] == 419:
        
        Q.append(i)

In [65]:
yandex_orders_metrics = yandex_orders_metrics.query('index not in @Q').reset_index(drop=True)

In [66]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [67]:
yandex_orders_metrics = yandex_orders_metrics.drop_duplicates().reset_index(drop=True)

In [68]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,Остаток,Цена
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,722
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,601
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,48,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,501
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,18,440
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590


In [69]:
# yandex_orders_metrics = yandex_orders_metrics.iloc[:, :-3]

In [70]:
for i in range(len(yandex_orders_metrics)):
    
    if len(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']])>1:
        display(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']])

In [71]:
if 'Остаток' in list(yandex_orders_metrics.columns):
    
    yandex_orders_metrics = yandex_orders_metrics.drop('Остаток', axis=1)
else:
    None

In [72]:
if 'Цена' in list(yandex_orders_metrics.columns):
    
    yandex_orders_metrics = yandex_orders_metrics.drop('Цена', axis=1)
else:
    None

In [73]:
yandex_orders_metrics = letter_substitution(yandex_orders_metrics)

In [74]:
try:
    yandex_orders_metrics.to_excel(path_algoritm + 'YANDEX_ORDERS/yandex_orders_metrics_истор..xlsx', index=False)
except:
    yandex_orders_metrics.to_excel(path_algoritm + 'YANDEX_ORDERS/yandex_orders_metrics_истор..xlsx', index=False)

In [75]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [76]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Загрузка и предобработка ежедневных YANDEX заказов

## Сведение в одну таблицу общих метрик и заказов за день

In [77]:
yandex_orders_new = None
df_for_date = None
day_filter = None
drop_status_orders = None
R = None

for path_file in P:
        
    yandex_orders_new = pd.read_excel(path_file, 
                                      sheet_name='Транзакции по заказам и товарам',
                                      skiprows=9,
                                      usecols=[9, 11, 13, 23],
                                      names=['Дата', 'Артикул', 'Количество', 'Статус'], 
                                      dtype={'Артикул': 'str', 'Статус': 'str'})

    df_for_date = pd.read_excel(path_file, 
                                sheet_name='Сводка', 
                                usecols=[0]).iloc[0:0]

    day_filter = pd.to_datetime(','.join(re.findall('(\d+)', 
                                df_for_date.columns[0])[:3]), 
                                format=('%d,%m,%Y'))\
                                .strftime('%d-%B-%Y')

    drop_status_orders = [x for x in list(yandex_orders_new['Статус']\
                                          .unique()) if 'отмен' in x or 'Отмен' in x or 'ОТМЕН' in x]

    yandex_orders_new = yandex_orders_new\
                        .query("Статус not in @drop_status_orders")\
                        .reset_index(drop=True)

    yandex_orders_new['Дата'] = [pd.to_datetime(x, format=('%d.%m.%Y'))\
                                .strftime('%d-%B-%Y') for x in list(yandex_orders_new['Дата'])]

    yandex_orders_new = yandex_orders_new[yandex_orders_new['Дата']==day_filter].reset_index(drop=True)

    yandex_orders_new = yandex_orders_new[['Артикул', 'Количество']]
    yandex_orders_new = letter_substitution(yandex_orders_new)
    yandex_orders_new = yandex_orders_new.groupby('Артикул')['Количество'].sum().reset_index(drop=False)
    yandex_orders_new.columns = ['Артикул', day_filter]
    if len(yandex_orders_new) - len(list(set(yandex_orders_new['Артикул']))) !=0:
        print('error')
        int('error')
    else:
        print(yandex_orders_new[day_filter].sum(), day_filter)

    try:
        yandex_orders_metrics
    except:
        None



    try:
        yandex_orders_new
    except:
        None

    try:
        yandex_orders_new[day_filter].sum()
    except:
        None

    if len(yandex_orders_new) - len(list(set(yandex_orders_new['Артикул']))) != 0:
        exit()

    yandex_orders_new = letter_substitution(yandex_orders_new)

    list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(yandex_orders_new['Артикул'])))


    try:   
        if day_filter not in list(yandex_orders_metrics.columns):


            if len(yandex_orders_new) < len(yandex_orders_metrics):

                yandex_orders_metrics = yandex_orders_metrics.merge(yandex_orders_new, on='Артикул', how='left')

                yandex_orders_metrics = yandex_orders_metrics.fillna(0)
                for row in list(yandex_orders_metrics.columns)[3:]:
                    yandex_orders_metrics[row] = yandex_orders_metrics[row].astype('int')
            else:

                yandex_orders_metrics = yandex_orders_metrics.merge(yandex_orders_new, on='Артикул', how='right')

                yandex_orders_metrics = yandex_orders_metrics.fillna(0)
                for row in list(yandex_orders_metrics.columns)[3:]:
                    yandex_orders_metrics[row] = yandex_orders_metrics[row].astype('int')



        else:
            None

        day_filter = []
        Q = []
        w = list(yandex_orders_metrics.columns)[:3]



        for row in list(yandex_orders_metrics.columns)[3:]:
            R.append(datetime.datetime.strptime(row, '%d-%B-%Y'))
        R.sort()

        for row in R:
            Q.append(row.strftime('%d-%B-%Y'))
        yandex_orders_metrics = yandex_orders_metrics[w + Q]
    except:
        None


    yandex_orders_metrics = yandex_orders_metrics[list(yandex_orders_metrics.columns[:3])\
                            + [y.strftime('%d-%B-%Y') for y in\
                            sorted([pd.to_datetime(x) for x in dt_col(yandex_orders_metrics)])]]

    yandex_orders_new = None
    df_for_date = None
    day_filter = None
    drop_status_orders = None
    R = None



/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1417 08-April-2024


In [78]:
wwm.unique_items(yandex_orders_metrics)

## Загрузка и обработка Остатков

In [79]:
yandex_prod_tail = pd.read_excel(path_file2)

In [80]:
yandex_prod_tail = yandex_prod_tail[[yandex_prod_tail.columns[0]]+[yandex_prod_tail.columns[2]]+[yandex_prod_tail.columns[1]]+[yandex_prod_tail.columns[5]]]

yandex_prod_tail = yandex_prod_tail.loc[4:]
yandex_prod_tail = yandex_prod_tail.rename(columns={yandex_prod_tail.columns[0]: 'Артикул',
                                                    yandex_prod_tail.columns[1]: 'Название',
                                                    yandex_prod_tail.columns[2]: 'Номенклатура',
                                                    yandex_prod_tail.columns[3]: 'Остаток'})
yandex_prod_tail = yandex_prod_tail.fillna(0)

yandex_prod_tail[list(yandex_prod_tail.columns[:2])] = yandex_prod_tail[list(yandex_prod_tail.columns[:2])].astype(str)                                                
yandex_prod_tail[list(yandex_prod_tail.columns[2:])] = yandex_prod_tail[list(yandex_prod_tail.columns[2:])].astype(int)

yandex_prod_tail = letter_substitution(yandex_prod_tail)
yandex_prod_tail2 = yandex_prod_tail.copy()

yandex_prod_tail = yandex_prod_tail.pivot_table(index='Артикул', values='Остаток', aggfunc='sum')
yandex_prod_tail['Артикул'] = list(yandex_prod_tail.index)
yandex_prod_tail.reset_index(drop=True, inplace=True)
yandex_prod_tail = yandex_prod_tail[['Артикул', 'Остаток']]

In [81]:
yandex_prod_tail['Остаток'].sum()

65692

In [82]:
yandex_prod_tail[yandex_prod_tail['Артикул']=='20503']

,Артикул,Остаток
92,20503,193


In [83]:
len(yandex_prod_tail) - len(list(set(yandex_prod_tail['Артикул'])))

0

In [84]:
yandex_prod_tail = letter_substitution(yandex_prod_tail)

In [85]:
len(yandex_prod_tail) - len(list(set(yandex_prod_tail['Артикул'])))

0

In [86]:
Q = []

for i in range(len(yandex_prod_tail)):
    
    row = yandex_prod_tail.loc[i]['Артикул']
    
    if len(yandex_prod_tail[yandex_prod_tail['Артикул']==row]) > 1 and yandex_prod_tail.loc[i]['Остаток'] == 0:
        
        Q.append(row)
        
Q = list(set(Q))
e = list(yandex_prod_tail.query('Артикул not in @Q').index)

w = []
for row in Q:
    
    w.append(yandex_prod_tail[yandex_prod_tail['Артикул']==row].index[0])
    
q = w+e

yandex_prod_tail = yandex_prod_tail.query('index in @q').reset_index(drop=True)
    

In [87]:
yandex_prod_tail

,Артикул,Остаток
0,16920,0
1,16930,124
2,16931,192
3,16932,0
4,16950,114
...,...,...
862,Z0247,0
863,Z0400,66
864,Z0401,89
865,Z0500,0


In [88]:
len(yandex_prod_tail) - len(list(set(yandex_prod_tail['Артикул'])))

0

In [89]:
yandex_prod_tail = yandex_prod_tail.drop_duplicates().reset_index(drop=True)

In [90]:
len(yandex_prod_tail) - len(list(set(yandex_prod_tail['Артикул'])))

0

In [91]:
list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(yandex_prod_tail['Артикул'])))

[]

In [92]:
len(yandex_prod_tail) - len(list(set(yandex_prod_tail['Артикул'])))

0

In [93]:
yandex_prod_tail['Остаток'].sum()

65692

In [94]:
yandex_prod_tail2

,Артикул,Название,Номенклатура,Остаток
0,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
1,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
2,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
3,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
4,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
...,...,...,...,...
3921,A8301,Набор емкостей для специй на вращающейся подст...,101948978735,0
3922,A8501,"Скалка для теста, Sugar Day, Скалка с регулиро...",101256533614,0
3923,A8501,"Скалка для теста, Sugar Day, Скалка с регулиро...",101256533614,0
3924,A8501,"Скалка для теста, Sugar Day, Скалка с регулиро...",101256533614,0


In [95]:
yandex_prod_tail2 = yandex_prod_tail2.drop_duplicates().reset_index(drop=True)

In [96]:
len(yandex_prod_tail2) - len(list(set(yandex_prod_tail2['Артикул'])))

1609

In [97]:
yandex_prod_tail2 = yandex_prod_tail2.drop_duplicates().reset_index(drop=True)
Q = list(set(yandex_prod_tail['Артикул']))
yandex_prod_tail2 = yandex_prod_tail2.query('Артикул in @Q').reset_index(drop=True)


        
        
        


In [98]:
Q = []

for i in range(len(yandex_prod_tail2)):
    
    if len(yandex_prod_tail2[yandex_prod_tail2['Артикул']==yandex_prod_tail2.loc[i]['Артикул']]) > 1:
        
        Q.append(yandex_prod_tail2.loc[i]['Артикул'])
        
Q = list(set(Q))


    
# yandex_prod_tail2 = yandex_prod_tail2.query('index in @R').reset_index(drop=True)
    

In [99]:
R = []
for row in Q:    
    for r in list(yandex_prod_tail2[yandex_prod_tail2['Артикул']==row].index[1:]):
        
        R.append(r)
     
    

In [100]:
yandex_prod_tail2 = yandex_prod_tail2.query('index not in @R').reset_index(drop=True)

In [101]:
yandex_prod_tail2

,Артикул,Название,Номенклатура,Остаток
0,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
1,16930,"Разделители для ящиков, Birdhouse, Раздвижные...",101827067058,57
2,16931,Полка в шкаф раздвижная Birdhouse/ Полочка доп...,102284457486,0
3,16932,Полка в шкаф раздвижная Birdhouse/ Полочка доп...,102284332300,0
4,16950,"Подсвечники для свечей металлические золотые, ...",101827065370,85
...,...,...,...,...
862,A7001,"Овощерезка ручная, Shiny Kitchen, Измельчитель...",101256518324,2
863,A7301,"Венчик для взбивания, Sugar Day, Венчик метал...",101255839694,0
864,A7501,"Форма для выпечки, Sugar Day, Форма для запека...",102216021673,28
865,A8301,Набор емкостей для специй на вращающейся подст...,101948978735,169


In [102]:
yandex_prod_tail2 = yandex_prod_tail2.drop_duplicates().reset_index(drop=True)

In [103]:
len(yandex_prod_tail2) - len(list(set(yandex_prod_tail2['Артикул'])))

0

In [104]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
yandex_prod_tail2

,Артикул,Название,Номенклатура,Остаток
0,16920,"Канцелярский нож скальпель, Parchment, Макетны...",101827068308,0
1,16930,"Разделители для ящиков, Birdhouse, Раздвижные...",101827067058,57
2,16931,Полка в шкаф раздвижная Birdhouse/ Полочка доп...,102284457486,0
3,16932,Полка в шкаф раздвижная Birdhouse/ Полочка доп...,102284332300,0
4,16950,"Подсвечники для свечей металлические золотые, ...",101827065370,85
...,...,...,...,...
862,A7001,"Овощерезка ручная, Shiny Kitchen, Измельчитель...",101256518324,2
863,A7301,"Венчик для взбивания, Sugar Day, Венчик метал...",101255839694,0
864,A7501,"Форма для выпечки, Sugar Day, Форма для запека...",102216021673,28
865,A8301,Набор емкостей для специй на вращающейся подст...,101948978735,169


## Сравнение с уникальными артикулами по дню и возможное добавление уникальных артикулов в "метрики"

In [106]:
# Все элементы первого массива, которых нет во втором 

main_list = np.setdiff1d(list(set(yandex_prod_tail2['Артикул'])), list(set(yandex_orders_metrics['Артикул'])))

if len(main_list) > 0:
    
    yandex_prod_tail2 = yandex_prod_tail2.query('Артикул in @main_list').iloc[:, :-1].reset_index(drop=True)
    yandex_orders_metrics = pd.concat([yandex_orders_metrics, yandex_prod_tail2], ignore_index=True)
    yandex_orders_metrics = yandex_orders_metrics.fillna(0)
    yandex_orders_metrics.iloc[:, 3:] = yandex_orders_metrics.iloc[:, 3:].astype(int)
    
else:
    
    None

In [107]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
yandex_orders_metrics.isna().sum()

Артикул            0
Название           0
Номенклатура       0
07-January-2022    0
08-January-2022    0
                  ..
04-April-2024      0
05-April-2024      0
06-April-2024      0
07-April-2024      0
08-April-2024      0
Length: 826, dtype: int64

In [109]:
yandex_orders_metrics[list(yandex_orders_metrics.columns[:2])] = yandex_orders_metrics[list(yandex_orders_metrics.columns[:2])].astype(str)


## Сведение метрик и остатков 

In [110]:
yandex_orders_metrics = yandex_orders_metrics.merge(yandex_prod_tail, on='Артикул', how='left')
yandex_orders_metrics = yandex_orders_metrics.fillna(0)
for row in list(yandex_orders_metrics.columns)[3:]:
    yandex_orders_metrics[row] = yandex_orders_metrics[row].astype('int')

In [111]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,18
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
yandex_orders_metrics = letter_substitution(yandex_orders_metrics)



In [113]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [114]:
price

,Артикул,Цена
0,16801,649
1,16802,649
2,16803,722
3,16804,601
4,16920,184
...,...,...
1145,A8202,369
1146,A8203,369
1147,A8301,1880
1148,A8501,590


In [115]:
yandex_orders_metrics = yandex_orders_metrics.merge(price, on='Артикул', how='left')
yandex_orders_metrics = yandex_orders_metrics.fillna(0)

In [116]:
yandex_orders_metrics.duplicated().sum()

0

In [117]:
yandex_orders_metrics = yandex_orders_metrics.drop_duplicates().reset_index(drop=True)

In [118]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,722
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,601
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,48,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,501
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,18,440
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,590


In [119]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [120]:
for i in range(len(yandex_orders_metrics)):
    
    if len(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']]) >1:
        
        display(yandex_orders_metrics[yandex_orders_metrics['Артикул']==yandex_orders_metrics.loc[i]['Артикул']])

## Выгрузка финальной таблицы YANDEX заказы

In [121]:
yandex_orders_metrics = wwm.rename_data_from_reestr(yandex_orders_metrics, 
                                                  'Название', 
                                                  reestr=reestr)

In [122]:
yandex_orders_metrics.to_csv(path_file3, index=False, sep=";", encoding='utf-8-sig')

In [123]:
cp = pd.read_excel(path_cost_price)
cp['Артикул'] = cp['Артикул'].astype(str)
cp['Себестоимость'] = cp['Себестоимость'].astype(float)

In [124]:
len(p) - len(list(set(p['Артикул'])))

0

In [125]:
len(cp) - len(list(set(cp['Артикул'])))

0

In [126]:
obvy['Артикул'] = obvy['Артикул'].astype(str)

In [127]:
len(obvy) - len(list(set(obvy['Артикул'])))

0

In [128]:
F = []
for i in range(len(obvy)):
    art = obvy.loc[i]['Артикул']
    if len(obvy[obvy['Артикул']==art]) > 1:
        F.append([art, sorted(list(obvy[obvy['Артикул']==art]['Объемный вес, кг']))[0]])
for row in F:
    obvy = obvy[(obvy['Артикул'] != row[0]) & (obvy['Объемный вес, кг'] != row[1])]
obvy = obvy.reset_index(drop=True)

In [129]:
for row in F:
    obvy = obvy[(obvy['Артикул'] != row[0]) & (obvy['Объемный вес, кг'] != row[1])]

In [130]:
obvy = obvy.reset_index(drop=True)

In [131]:
len(obvy) - len(list(set(obvy['Артикул'])))

0

In [132]:
p

,Артикул,размещение на витрине %
0,19600,15.0
1,20501,10.0
2,20503,10.0
3,27700,15.0
4,27701,15.0
...,...,...
824,G0504,13.5
825,Z0501,17.0
826,G0142,16.5
827,27750,18.0


In [133]:
p

,Артикул,размещение на витрине %
0,19600,15.0
1,20501,10.0
2,20503,10.0
3,27700,15.0
4,27701,15.0
...,...,...
824,G0504,13.5
825,Z0501,17.0
826,G0142,16.5
827,27750,18.0


In [134]:
yandex_orders_metrics = yandex_orders_metrics.merge(cp, on='Артикул', how='left')

In [135]:
# yandex_orders_metrics = yandex_orders_metrics.merge(p, on='Артикул', how='left')

In [136]:
yandex_orders_metrics['размещение на витрине %'] = 4
p['размещение на витрине %'] = 4

In [137]:
yandex_orders_metrics = yandex_orders_metrics.merge(obvy, on='Артикул', how='left')

In [138]:
yandex_orders_metrics

,Артикул,Название,Номенклатура,07-January-2022,08-January-2022,09-January-2022,10-January-2022,11-January-2022,12-January-2022,13-January-2022,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Себестоимость,размещение на витрине %,"Объемный вес, кг"
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,1.019762e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,649,243.71,4,1.588
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,649,243.71,4,1.588
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,1.013120e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,722,243.71,4,1.588
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,1.020809e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,601,243.71,4,1.588
4,18301,"Чехол для airpods 2 с карабином, белый",1.013094e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,48,151,50.42,4,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,50976,"Зеркальная наклейка шестигранная, 24шт",1.028358e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,501,91.85,4,0.147
1151,50975,"Наклейки интерьерные, кольца",1.028419e+11,0,0,0,0,0,0,0,...,0,2,0,0,0,18,440,73.80,4,0.120
1152,50456,"Расческа с зеркалом матовая, розовая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,590,74.78,4,0.070
1153,50457,"Расческа с зеркалом матовая, зеленая",1.029119e+11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,590,74.78,4,0.070


In [139]:
yandex_orders_metrics_AC = yandex_orders_metrics.copy()
yandex_orders_metrics_AC = yandex_orders_metrics_AC.drop('Цена', axis=1)

In [140]:
def razm_v_rub(data):
    
    if 'размещение в рублях' not in list(data.columns):
    
#         Q = []
#         for i in range(len(data)):
#             if data.loc[i]['Цена'] > 0:
#                 if data.loc[i]['Цена'] < min_price:
#                     if data.loc[i]['Цена'] >= prazm_v_rub_porog_max:
#                         Q.append(tariff_max)
#                     elif data.loc[i]['Цена'] >= razm_v_rub_porog_min:
#                         Q.append(tariff_mean)
#                     else:
#                         Q.append(tariff_min)
#                 else:
#                     if str(data.loc[i]['размещение на витрине %']) != 'nan':
#                         Q.append(data.loc[i]['Цена']/100*data.loc[i]['размещение на витрине %'])
#                     else:
#                         Q.append(None)                


#         data['размещение в рублях'] = Q


        data['размещение в рублях'] = data['Цена'] / 100 * data['размещение на витрине %'] 
        
    else:
        None
    
    return data
                         

In [141]:
# def sklad_obr(data):
    
#     if 'складская обработка' not in list(data.columns):
        
#         Q = []
#         for i in range(len(data)):
#             if data.loc[i]['Цена'] > 0:
#                 if data.loc[i]['Цена'] < min_price:
#                     Q.append(0)
#                 else:
#                     SO = data.loc[i]['Цена'] / 100 * proc_sklad_obr
#                     if SO < sklad_obr_porog_min:
#                         Q.append(sklad_obr_porog_min)
#                     elif SO > sklad_obr_porog_max:
#                         Q.append(sklad_obr_porog_max)
#                     else:
#                         Q.append(SO)
#             else:
#                 Q.append(None)

#         data['складская обработка'] = Q
        
#     else:
#         None
    
#     return data 


In [142]:
def sklad_obr(data):
    
    if 'складская обработка' not in list(data.columns):
        
        Q = []
        for i in range(len(data)):
            SO = data.loc[i]['Цена'] / 100 * proc_sklad_obr
            if SO < sklad_obr_porog_min:
                Q.append(sklad_obr_porog_min)
            elif SO > sklad_obr_porog_max:
                Q.append(sklad_obr_porog_max)
            else:
                Q.append(SO)

        data['складская обработка'] = Q
        
    else:
        
        None
    
    return data 


In [143]:
def dost_FO(data):
    
    if 'доставка в ФО' not in list(data.columns):
    
        Q = []
        for i in range(len(data)):
            if data.loc[i]['Цена'] > 0:
                if data.loc[i]['Объемный вес, кг'] < 0.2:
                    Q.append(10)
                elif data.loc[i]['Объемный вес, кг'] < 0.5:
                    Q.append(20)
                elif data.loc[i]['Объемный вес, кг'] < 1:
                    Q.append(30)
                elif data.loc[i]['Объемный вес, кг'] < 2:
                    Q.append(40)
                elif data.loc[i]['Объемный вес, кг'] < 5:
                    Q.append(60)
                elif data.loc[i]['Объемный вес, кг'] < 6:
                    Q.append(80)
                elif data.loc[i]['Объемный вес, кг'] < 8:
                    Q.append(100)
                elif data.loc[i]['Объемный вес, кг'] < 10:
                    Q.append(120)
                elif data.loc[i]['Объемный вес, кг'] < 12:
                    Q.append(200)
                elif data.loc[i]['Объемный вес, кг'] < 15:
                    Q.append(250)
                elif data.loc[i]['Объемный вес, кг'] < 20:
                    Q.append(325)
                elif data.loc[i]['Объемный вес, кг'] < 25:
                    Q.append(375)
                elif data.loc[i]['Объемный вес, кг'] < 30:
                    Q.append(400)
                elif data.loc[i]['Объемный вес, кг'] < 35:
                    Q.append(500)
                elif data.loc[i]['Объемный вес, кг'] > 35:
                    Q.append(550)
                else:
                    Q.append(None)
            else:
                Q.append(None)

        data['доставка в ФО'] = Q
    
    else:
        None
    
    return data
        
        

In [144]:
def posl_mil(data):
    
    if 'последняя миля' not in list(data.columns):
    
        Q = []
        
        for i in range(len(data)):
            
            PM = data.loc[i]['Цена'] / 100 * proc_posl_mil
            if PM < posl_mil_porog_min:
                Q.append(posl_mil_porog_min)
            elif PM > posl_mil_porog_max:
                Q.append(posl_mil_porog_max)
            else:
                Q.append(PM)
                
        data['последняя миля'] = Q
        
    else:
        None

    return data 


In [145]:
def NDS(data):
    
    if 'НДС' not in list(data.columns):
        
        data['Цена_НДС'] = data['Цена'] * 0.2 / 1.2
        data['Себестоимость_НДС'] = data['Себестоимость'] * 0.2 / 1.2
        data['размещение_НДС'] = data['размещение в рублях'] * 0.2 / 1.2
        data['складская_обработка_НДС'] = data['складская обработка'] * 0.2 / 1.2 
        data['доставка_НДС'] = data['доставка'] * 0.2 / 1.2 
        data['прием_платежей_НДС'] = data['прием платежей'] * 0.2 / 1.2 
        data['программа_лояльности_НДС'] = data['программа лояльности'] * 0.2 / 1.2
        
        data['НДС'] = (data['Цена_НДС'] - 
                       data['Себестоимость_НДС'] - 
                       data['размещение_НДС'] - 
                       data['складская_обработка_НДС'] - 
                       data['доставка_НДС'] - 
                       data['прием_платежей_НДС'] - 
                       data['программа_лояльности_НДС'])
        
    else:
        None
        
    return data

In [146]:
def NNP(data):
    
    if 'ННП' not in list(data.columns):
        
        data['Цена_ННП'] = data['Цена'] - data['Цена_НДС']
        data['Себестоимость_ННП'] = data['Себестоимость'] - data['Себестоимость_НДС']
        data['размещение_ННП'] = data['размещение в рублях'] - data['размещение_НДС']
        data['складская_обработка_ННП'] = data['складская обработка'] - data['складская_обработка_НДС']
        data['доставка_ННП'] = data['доставка'] - data['доставка_НДС']
        data['прием_платежей_ННП'] = data['прием платежей'] - data['прием_платежей_НДС'] 
        data['программа_лояльности_ННП'] = data['программа лояльности'] - data['программа_лояльности_НДС']
        
        data['ННП'] = (data['Цена_ННП'] - 
                       data['Себестоимость_ННП'] - 
                       data['размещение_ННП'] - 
                       data['складская_обработка_ННП'] - 
                       data['доставка_ННП'] - 
                       data['прием_платежей_ННП'] - 
                       data['программа_лояльности_ННП']) * 0.2
        
    else:
        None
        
    return data

In [147]:
def profit(data):
    
    if 'Прибыль' not in list(data.columns):
        
        data['Прибыль'] = (data['Цена'] - 
                           data['Себестоимость'] - 
                           data['размещение в рублях'] - 
                           data['складская обработка'] - 
                           data['доставка'] - 
                           data['прием платежей'] - 
                           data['программа лояльности'] - data['Налог'])
    else:
        None
        
    return data

In [148]:
def margin(data):
    
    if 'Наценка' not in list(data.columns):
        
        data['Наценка'] = (data['Прибыль'] * 100 / data['Себестоимость']).round(2)
        
        
        
        
        
        
    else:
        None
        
    return data    

In [149]:
def calculated_features(data):
    
    data = razm_v_rub(data)
    data = sklad_obr(data)
    data = dost_FO(data)
    data = posl_mil(data)

    data['доставка'] = data['доставка в ФО'] + data['последняя миля']
    data['прием платежей'] = data['Цена'] / 100 * proc_priem_plat
    data['программа лояльности'] = data['Цена'] / 100 * proc_progr_lo

    data = NDS(data)
    data = NNP(data)
    data['Налог'] = data['НДС'] + data['ННП']
    data = profit(data)
    data = margin(data)
    
    drop_columns = ['размещение на витрине %', 'Объемный вес, кг',
       'размещение в рублях', 'складская обработка', 'доставка в ФО',
       'последняя миля', 'доставка', 'прием платежей', 'программа лояльности',
       'Цена_НДС', 'Себестоимость_НДС', 'размещение_НДС',
       'складская_обработка_НДС', 'доставка_НДС', 'прием_платежей_НДС',
       'программа_лояльности_НДС', 'НДС', 'Цена_ННП', 'Себестоимость_ННП',
       'размещение_ННП', 'складская_обработка_ННП', 'доставка_ННП',
       'прием_платежей_ННП', 'программа_лояльности_ННП', 'ННП', 'Налог',
       'Прибыль']
    
    data = data.drop(drop_columns, axis=1)
    
    return data

In [150]:
def calculated_features_v2(data):
    
    data = razm_v_rub(data)
    data = sklad_obr(data)
    data = dost_FO(data)
    data = posl_mil(data)

    data['доставка'] = data['доставка в ФО'] + data['последняя миля']
    data['прием платежей'] = data['Цена'] / 100 * proc_priem_plat
    data['программа лояльности'] = data['Цена'] / 100 * proc_progr_lo

    data = NDS(data)
    data = NNP(data)
    data['Налог'] = data['НДС'] + data['ННП']
    data = profit(data)
    data = margin(data)
    
    
    return data

In [151]:
def drop_cols(data):
    
    drop_columns = ['размещение на витрине %', 'Объемный вес, кг',
       'размещение в рублях', 'складская обработка', 'доставка в ФО',
       'последняя миля', 'доставка', 'прием платежей', 'программа лояльности',
       'Цена_НДС', 'Себестоимость_НДС', 'размещение_НДС',
       'складская_обработка_НДС', 'доставка_НДС', 'прием_платежей_НДС',
       'программа_лояльности_НДС', 'НДС', 'Цена_ННП', 'Себестоимость_ННП',
       'размещение_ННП', 'складская_обработка_ННП', 'доставка_ННП',
       'прием_платежей_ННП', 'программа_лояльности_ННП', 'ННП', 'Налог',
       'Прибыль']
    
    data = data.drop(drop_columns, axis=1)
    
    return data

In [152]:
nac_flag 

False

In [153]:
if nac_flag == True:
    art = '22800'
    price = 1099
    yandex_orders_metrics2 = yandex_orders_metrics[yandex_orders_metrics['Артикул']==art].copy()
    yandex_orders_metrics2.loc[yandex_orders_metrics2.index[0], 'Цена'] = price
    calculated_features_v2(yandex_orders_metrics2.reset_index(drop=True))[['Артикул', 'Цена', 'Наценка']]
    exit()

In [154]:
yandex_orders_metrics = calculated_features_v2(yandex_orders_metrics)

In [155]:
rc = pd.read_excel(path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx')

In [156]:
rc['Артикул'] = rc['Артикул'].astype(str)

In [157]:



yandex_orders_metrics = yandex_orders_metrics.merge(rc, on='Артикул', how='left')

yandex_orders_metrics['Остаток на РЦ'] = yandex_orders_metrics['Остаток на РЦ'].fillna(0)

In [158]:
yandex_orders_metrics['Наценка'] = yandex_orders_metrics['Наценка'].round(2)

In [159]:
yandex_orders_metrics_11 = yandex_orders_metrics.copy()

In [160]:
yandex_orders_metrics = drop_cols(yandex_orders_metrics)

In [161]:
yandex_orders_metrics = yandex_orders_metrics[['Артикул', 'Название', 'Номенклатура'] + dt_col(yandex_orders_metrics) + ['Остаток', 'Цена', 'Наценка', 'Остаток на РЦ']]



In [162]:
def weekly_amount(data):
    
    data1 = data.copy()

    z = list(data.iloc[:, 3:-5].columns)

    for i in range(len(z)):
        if pd.to_datetime(z[i]).dayofweek == 6 and i > 6:
            data1.insert((data1.columns.get_loc(z[i]))+1, f"{pd.to_datetime(z[i-6]):%d-%B}-{pd.to_datetime(z[i]):%d-%B}", 0)



    data = data1

    R = []
    for i in range(len(list(data.columns))):
        try:
            if len((re.findall('(\d+)',list(data.columns)[i]))[1]) != 4:
                R.append(list(data.columns)[i])

        except:
            None

    for row in R:
        T = []
        for i in range(len(data)):
            T.append((data.iloc[i, data.columns.get_loc(row)-7:data.columns.get_loc(row)]).sum())

        data[row] = T
        
    
    data['Артикул'] = data['Артикул'].astype(str)
        
    return data

In [163]:
len(yandex_orders_metrics) - len(list(set(yandex_orders_metrics['Артикул'])))

0

In [164]:
list(yandex_orders_metrics[yandex_orders_metrics['Артикул']=='A5501'].index.values)
yandex_orders_metrics = yandex_orders_metrics.drop(index=list(yandex_orders_metrics[yandex_orders_metrics['Артикул']=='A5501'].index.values)[1:]).reset_index(drop=True)

In [165]:
yandex_orders_metrics = yandex_orders_metrics.drop(index=list(yandex_orders_metrics[yandex_orders_metrics['Артикул']=='A5501'].index.values)[1:]).reset_index(drop=True)

In [166]:
yandex_orders_metrics['Наценка'].isna().sum()

18

In [167]:
yandex_orders_metrics['Номенклатура'] = yandex_orders_metrics['Номенклатура'].astype(int)

In [168]:
yandex_orders_metrics_22 = yandex_orders_metrics.copy()

## Выгрузка предобработанной таблицы (30 дней + "Итого")

In [169]:
df = yandex_orders_metrics.copy()
df = df.sort_values(by='Артикул')

df = df[list(df.columns[:3])+list(df.columns[-52:])]

df['      '] = '      '

df = df[list(df.columns[:51])+['      ']+list(df.columns[-5:-1])]


df2 = df.copy()
df2 = df2.loc[0:0]
df2['Артикул'] = 'Итого'
df2[list(df.columns)[1]] = None
df2[list(df.columns)[2]] = None
for row in list(df.columns)[3:-5]:
    df2[row] = sum(df[row])
    
df2['      '] = '      '
df2['Остаток'] = sum(df['Остаток'])
df2['Цена'] = sum(df['Цена'])
df2['Остаток на РЦ'] = sum(df['Остаток на РЦ'])
df2['Наценка'] = sum(df['Наценка'])
df = df.append(df2, ignore_index=True)

/var/folders/kd/z6thdxpn5wgf06_21113yy0c0000gq/T/ipykernel_39920/2129236102.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


In [170]:
df = weekly_amount(df)

In [171]:
len(df) - len(list(set(df['Артикул'])))

0

In [172]:
df = df.merge(fbs_stocks, on='Артикул', how='left')

### Добавление расчетных промежуточных признаков 

## Добавление статуса 

In [173]:
st = pd.read_excel(path_st)
Q = []

for i in range(len(df)):
    row = df.loc[i]['Артикул']
    if df.loc[i]['Артикул'] in list(set(st['Артикул'])):
                                
        Q.append(st[st['Артикул']==row]['Статус'].values[0])
        
        
    else:
        Q.append('статус не определен')
        
Q[-1] = None

df['Статус'] = Q


df = df[list(df.columns[:3]) + ['Статус'] + list(df.columns[3:-1])]

In [174]:
df = df.drop_duplicates().reset_index(drop=True)

In [175]:
df['                      '] = None

In [176]:
df[df['Наценка']==np.inf]

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,Остаток fbs,
598,A5901,"Тендерайзер, белый",101255840708,архив,0,0,0,0,0,0,...,0,0,0,,0,199,inf,0.0,0.0,None
599,A5902,"Тендерайзер, черный",101255859666,отказ,0,0,0,0,0,0,...,0,0,0,,0,199,inf,1.0,0.0,None
603,A6601,Набор кухонных принадлежностей из силикона и б...,101255866220,архив,0,0,0,0,0,0,...,0,0,0,,0,2129,inf,0.0,0.0,None
622,A7602,"Силиконовая форма для выпечки, мягкий квадрат",101309645055,архив,0,0,0,0,0,0,...,0,0,0,,0,365,inf,0.0,0.0,None
1035,H0200,Пробковый блок,101682464878,архив,0,0,0,0,0,0,...,0,0,0,,0,295,inf,0.0,0.0,None


In [177]:
dfn = None

dfn = yandex_orders_metrics_11[['Артикул', 'Прибыль', 'Себестоимость', 'Наценка', dt_col(yandex_orders_metrics_11)[-1]]]\
     .rename(columns={dt_col(yandex_orders_metrics_11)[-1]: 'Продажи'})

dfn = dfn[(dfn['Себестоимость'].astype(str).isin(['0', 'nan', '0.0'])==False)\
        &(dfn['Продажи']>0)]\
     .reset_index(drop=True)

ITOG_NAC = round((((dfn['Прибыль'] * dfn['Продажи']).sum() / (dfn['Себестоимость'] *  dfn['Продажи']).sum()) * 100), 2)

dfn = None

In [178]:
ITOG_NAC

89.74

In [179]:
df['Наценка'] = df['Наценка'].where(df['Наценка']!=np.inf, np.nan)

df['Выручка'] = df['Цена'] * df[sorted([pd.to_datetime(x) for x in dt_col(df)])[-1].strftime('%d-%B-%Y')] 
df['Выручка'].loc[len(df)-1] = 0
df['Выручка'] = df['Выручка'].astype(float)
s = df['Выручка'].sum()
df['Доля выручки'] = (df['Выручка'] * 100) / s
df['Выручка'].loc[len(df)-1] = s
df['Итого наценка'] = None
df['Итого наценка'].loc[len(df)-1] = ITOG_NAC

In [180]:
for i in range(len(df)):
    
    row = df.loc[i]['Артикул']
    
    if len(df[df['Артикул']==row]) > 1:
        
        display(df[df['Артикул']==row])

In [181]:
# df.duplicated().sum()

In [182]:
len(df) - len(list(set(df['Артикул'])))

0

In [183]:
yandex_orders_metrics_22 = yandex_orders_metrics_22[['Артикул', 'Название', 'Номенклатура'] + dt_col(yandex_orders_metrics_22)]
yandex_orders_metrics_33 = yandex_orders_metrics_11[['Артикул', 'Цена']]
yandex_orders_metrics_33.columns = ['Артикул', dt_col(yandex_orders_metrics_11)[-1]]
yandex_orders_metrics_44 = yandex_orders_metrics_11[['Артикул', 'Остаток']]
yandex_orders_metrics_44.columns = ['Артикул', dt_col(yandex_orders_metrics_11)[-1]]

In [184]:
for i in range(len(yandex_orders_metrics_22)):
    
    if len(yandex_orders_metrics_22[yandex_orders_metrics_22['Артикул']==yandex_orders_metrics_22.loc[i]['Артикул']]) > 1:
        
        display(yandex_orders_metrics_22[yandex_orders_metrics_22['Артикул']==yandex_orders_metrics_22.loc[i]['Артикул']])

In [185]:
for i in range(len(yandex_orders_metrics_33)):
    
    if len(yandex_orders_metrics_33[yandex_orders_metrics_33['Артикул']==yandex_orders_metrics_33.loc[i]['Артикул']]) > 1:
        
        display(yandex_orders_metrics_33[yandex_orders_metrics_33['Артикул']==yandex_orders_metrics_33.loc[i]['Артикул']])

In [186]:
for i in range(len(yandex_orders_metrics_44)):
    
    if len(yandex_orders_metrics_44[yandex_orders_metrics_44['Артикул']==yandex_orders_metrics_44.loc[i]['Артикул']]) > 1:
        
        display(yandex_orders_metrics_44[yandex_orders_metrics_44['Артикул']==yandex_orders_metrics_44.loc[i]['Артикул']])

In [187]:
ya_price_metrics = pd.read_excel(path_ya_price_metrics)
ya_price_metrics['Артикул'] = ya_price_metrics['Артикул'].astype(str)

stocks_ya_metrics = pd.read_excel(path_stocks_ya_metrics)
stocks_ya_metrics['Артикул'] = stocks_ya_metrics['Артикул'].astype(str)


In [188]:
ya_price_metrics[list(ya_price_metrics.columns[:1])\
+ [y.strftime('%d-%B-%Y') for y in\
sorted([pd.to_datetime(x) for x in dt_col(ya_price_metrics)])]]\
.to_excel(path_ya_price_metrics, index=False)


stocks_ya_metrics[list(stocks_ya_metrics.columns[:1])\
+ [y.strftime('%d-%B-%Y') for y in\
sorted([pd.to_datetime(x) for x in dt_col(stocks_ya_metrics)])]]\
.to_excel(path_stocks_ya_metrics, index=False)


In [189]:
for i in range(len(ya_price_metrics)):
    
    if len(ya_price_metrics[ya_price_metrics['Артикул']==ya_price_metrics.loc[i]['Артикул']]) > 1:
        
        display(ya_price_metrics[ya_price_metrics['Артикул']==ya_price_metrics.loc[i]['Артикул']])

In [190]:
for i in range(len(stocks_ya_metrics)):
    
    if len(stocks_ya_metrics[stocks_ya_metrics['Артикул']==stocks_ya_metrics.loc[i]['Артикул']]) > 1:
        
        display(stocks_ya_metrics[stocks_ya_metrics['Артикул']==stocks_ya_metrics.loc[i]['Артикул']])

In [191]:
len(ya_price_metrics) - len(list(set(ya_price_metrics['Артикул'])))

0

In [192]:
len(stocks_ya_metrics) - len(list(set(stocks_ya_metrics['Артикул'])))

0

In [193]:
len(yandex_orders_metrics_33) - len(list(set(yandex_orders_metrics_33['Артикул'])))

0

In [194]:
len(yandex_orders_metrics_44) - len(list(set(yandex_orders_metrics_44['Артикул'])))

0

In [195]:
dt_col(yandex_orders_metrics_33)[-1]

'08-April-2024'

In [196]:
if dt_col(yandex_orders_metrics_33)[-1] not in list(ya_price_metrics.columns):
    
    ya_price_metrics = ya_price_metrics.merge(yandex_orders_metrics_33, on='Артикул', how='outer')
    
else:
    None


if dt_col(yandex_orders_metrics_44)[-1] not in list(stocks_ya_metrics.columns):
    
    stocks_ya_metrics = stocks_ya_metrics.merge(yandex_orders_metrics_44, on='Артикул', how='outer')
    
else:
    None

In [197]:
ya_price_metrics['Артикул'] = ya_price_metrics['Артикул'].astype(str)
stocks_ya_metrics['Артикул'] = stocks_ya_metrics['Артикул'].astype(str)

In [198]:
len(ya_price_metrics) - len(list(set(ya_price_metrics['Артикул'])))

0

In [199]:
len(stocks_ya_metrics) - len(list(set(stocks_ya_metrics['Артикул'])))

0

In [200]:
len(ya_price_metrics) - len(list(set(ya_price_metrics['Артикул'])))

0

In [201]:
len(stocks_ya_metrics) - len(list(set(stocks_ya_metrics['Артикул'])))

0

In [202]:
len(yandex_orders_metrics_11) - len(list(set(yandex_orders_metrics_11['Артикул'])))

0

In [203]:
ya_price_metrics.drop_duplicates().to_excel(path_ya_price_metrics, index=False)
stocks_ya_metrics.drop_duplicates().to_excel(path_stocks_ya_metrics, index=False)

In [204]:
ym = pd.read_excel(path_downloads + 'яндекс.xlsx')
ym['Артикул'] = ym['Артикул'].astype(str)
ym = ym[['Артикул', 'новинка']]

In [205]:
len(df) - len(list(set(df['Артикул'])))

0

In [206]:
# len(dop22) - len(list(set(dop22['Артикул'])))

In [207]:
len(ym) - len(list(set(ym['Артикул'])))

0

In [208]:
df = df.merge(ym, on='Артикул', how='left')

In [209]:
len(df) - len(list(set(df['Артикул'])))

0

In [210]:
# len(dop22) - len(list(set(dop22['Артикул'])))

In [211]:
df

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,Остаток,Цена,Наценка,Остаток на РЦ,Остаток fbs,,Выручка,Доля выручки,Итого наценка,новинка
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,101976154101,отказ,0,0,0,0,0,0,...,0,649,73.50,3.0,0.0,None,0.0,0.000000,None,NaN
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945750,отказ,0,0,0,0,0,0,...,0,649,73.50,4.0,0.0,None,0.0,0.000000,None,NaN
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,101312041369,отказ,0,0,0,0,0,0,...,0,722,90.66,4.0,0.0,None,0.0,0.000000,None,NaN
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945752,архив,0,0,0,0,0,0,...,0,601,61.92,0.0,0.0,None,0.0,0.000000,None,NaN
4,16920,Нож канцелярский металлический,101827068308,текущий,0,0,0,0,0,0,...,0,184,135.90,0.0,0.0,None,0.0,0.000000,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,Z0401,Ваза для цветов и сухоцветов решетка,101986518829,текущий Дарья,0,0,0,3,1,1,...,89,1380,67.31,0.0,53.0,None,2760.0,0.331799,None,NaN
1152,Z0402,Ваза 5 колб,102591438907,архив,0,0,0,0,0,0,...,0,3900,NaN,0.0,NaN,None,0.0,0.000000,None,NaN
1153,Z0500,Декантер с бокалами,101994522856,текущий Дарья,0,0,0,0,1,0,...,0,3510,143.12,4.0,1.0,None,0.0,0.000000,None,NaN
1154,Z0501,Графин для воды,102071515515,текущий Дарья,0,1,0,0,0,0,...,0,1372,84.14,7.0,0.0,None,0.0,0.000000,None,NaN


### В резерве

In [212]:
vr = pd.read_excel(path_vr)
vr['Артикул'] = vr['Артикул'].astype(str)

df = df.merge(vr, on='Артикул', how='left')

df = df[list(df.columns[:df.columns.get_loc('Остаток на РЦ')+1]) + ['В резерве'] + list(df.columns[df.columns.get_loc('Остаток на РЦ')+1:-1])]

In [213]:
dict_for_metric_ya = {}
dict_for_metric_ya['Col_for_del'] = [x for x in list(yandex_orders_metrics_AC.columns) if 'Артикул' not in x]
pd.DataFrame(dict_for_metric_ya).to_excel(path_downloads + 'Col_for_del_ya.xlsx', index=False)
dict_for_metric_ya = None

In [214]:
rc_irc = list(df[['Артикул']]\
         .merge(pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
                dtype={'Артикул': 'str'}).groupby('Артикул')['Остаток РЦ Иркутск'].sum()\
                .reset_index(drop=False), 
                on='Артикул', 
                how='left')['Остаток РЦ Иркутск'])

df.insert(df.columns.get_loc('Остаток на РЦ')+1, 'Остаток РЦ Иркутск', rc_irc)

# Расчет наценки для акции

In [215]:
def itogo_nac_ya(data, name_AC):
    
    data2 = data.copy()

    data['Наценка' + f'_{name_AC}'] = data['Наценка' + f'_{name_AC}'].where(data['Наценка' + f'_{name_AC}']!=np.inf, np.nan)

    data['Выручка' + f'_{name_AC}'] = data['Цена' + f'_{name_AC}'] * data[sorted([pd.to_datetime(x) for x in dt_col(data)])[-1].strftime('%d-%B-%Y')] 
    data['Выручка' + f'_{name_AC}'].loc[len(data)-1] = 0
    data['Выручка' + f'_{name_AC}'] = data['Выручка' + f'_{name_AC}'].astype(float)
    s = data['Выручка' + f'_{name_AC}'].sum()
    data['Доля выручки' + f'_{name_AC}'] = (data['Выручка' + f'_{name_AC}'] * 100) / s
    data['Выручка' + f'_{name_AC}'].loc[len(data)-1] = s
    data['Итого наценка' + f'_{name_AC}'] = data['Доля выручки' + f'_{name_AC}'] / 100 * data['Наценка' + f'_{name_AC}']
    e = data['Итого наценка' + f'_{name_AC}'].sum()
    data['Итого наценка' + f'_{name_AC}'].loc[len(data)-1] = e

    data2['Итого наценка' + f'_{name_AC}'] = data['Итого наценка' + f'_{name_AC}']

    data, data2 = data2, None

    return data

In [216]:
df_columns_1 = list(df.columns)

In [217]:
if len(path_AC) > 0:
    df_price_for_dop_nac = df[['Артикул', 'Цена']].copy()
    for AC in path_AC:
        try:
            ya_AC = pd.read_excel(path_downloads + AC, sheet_name='Товары и цены')
            name_AC = ','.join(ya_AC.columns[0].split(':')[1:]) + '_идет'
            ya_AC = ya_AC.iloc[4:, :].reset_index(drop=True)
            ya_AC.columns = list(ya_AC.loc[0].values)

            if str(list(set(ya_AC.iloc[0, :]))[0]) == 'nan':
                ya_AC = pd.read_excel(path_downloads + AC, sheet_name='Товары и цены')
                name_AC = ','.join(ya_AC.columns[0].split(':')[1:]) + '_идет'
                ya_AC = ya_AC.iloc[5:, :].reset_index(drop=True)
                ya_AC.columns = list(ya_AC.loc[0].values)
            try:

                try:
                    ya_AC = ya_AC[['Ваш SKU', 'Цена по акции']]
                except:
                    ya_AC = ya_AC[['Ваш SKU', 'Максимальный порог цены для участия в акции']]
                    
            except:
                ya_AC = ya_AC[['Ваш SKU', 'Максимальная цена для участия в акции']]
            
            ya_AC.columns = ['Артикул', 'Цена']
            Q = []
            for i in range(len(ya_AC)):
                try:
                    int(ya_AC.loc[i]['Цена'])
                except:
                    Q.append(i)
            ya_AC = ya_AC.query('index not in @Q').reset_index(drop=True)
            ya_AC['Артикул'] = ya_AC['Артикул'].astype(str)
            ya_AC['Цена'] = ya_AC['Цена'].astype(int)
        except:
            print(AC)
            exit()
        if len(ya_AC) - len(list(set(ya_AC['Артикул']))) != 0:
            exit()
        else:
            df = df.merge(calculated_features(yandex_orders_metrics_AC.merge(ya_AC, on='Артикул', how='inner'))[['Артикул', 'Цена', 'Наценка']]\
                          .rename(columns={'Цена': 'Цена' + f'_{name_AC}',
                                           'Наценка': 'Наценка' + f'_{name_AC}'}), 
                          on='Артикул', 
                          how='left')
            df_price_for_dop_nac = df_price_for_dop_nac.merge(ya_AC.rename(columns={'Цена': 'Цена' + f'_{name_AC}'}), 
                                       on='Артикул', 
                                       how='left')
            df = itogo_nac_ya(df, name_AC=name_AC)

    Q = []
    for i in range(len(df_price_for_dop_nac)):
        Q.append(sorted([x for x in list(df_price_for_dop_nac.iloc[i, 1:]) if str(x) != 'nan'])[0])
    df_price_for_dop_nac = df_price_for_dop_nac[['Артикул']]
    df_price_for_dop_nac['Цена'] = Q    
    df_price_for_dop_nac.loc[len(df_price_for_dop_nac)-1, 'Цена'] = None
    name_AC = '_совокупная'
    df = df.merge(calculated_features(yandex_orders_metrics_AC.merge(df_price_for_dop_nac, on='Артикул', how='inner'))[['Артикул', 'Цена', 'Наценка']]\
                  .rename(columns={'Цена': 'Цена' + f'_{name_AC}',
                                   'Наценка': 'Наценка' + f'_{name_AC}'}), 
                  on='Артикул', 
                  how='left')
    df = itogo_nac_ya(df, name_AC=name_AC)
else:
    None

In [218]:
rr = list(df.columns)

In [219]:
price_for_PRICES = pd.read_excel(path_price_for_PRICES,
                                 skiprows=4,
                                 dtype={'SKU': 'str'}).iloc[1:, :]\
                                 .reset_index(drop=True)\
                                 .rename(columns={'SKU': 'Артикул'})

price_for_PRICES_himki = pd.read_excel(path_price_for_PRICES_himki,
                                       skiprows=4,
                                       dtype={'SKU': 'str'}).iloc[1:, :]\
                                       .reset_index(drop=True)\
                                       .rename(columns={'SKU': 'Артикул'})

price_for_PRICES_irkutsk = pd.read_excel(path_price_for_PRICES_irkutsk,
                                         skiprows=4,
                                         dtype={'SKU': 'str'}).iloc[1:, :]\
                                         .reset_index(drop=True)\
                                         .rename(columns={'SKU': 'Артикул'})

In [220]:
if len([x for x in list(price_for_PRICES['Со скидками за ваш счёт, ₽']) if str(x)!='nan']) > 0:
    df = df.merge(price_for_PRICES[['Артикул', 'Со скидками за ваш счёт, ₽']], on='Артикул', how='left')
    df['Цена с учетом скидок'] = df['Со скидками за ваш счёт, ₽']
    df = df.merge(calculated_features(yandex_orders_metrics_AC.merge(price_for_PRICES[['Артикул', 
                                                                                       'Со скидками за ваш счёт, ₽']]\
                                                       .rename(columns={'Со скидками за ваш счёт, ₽': 'Цена'})\
                                                       .query('Цена > 0').reset_index(drop=True), 
                                                       on='Артикул', 
                                                       how='inner'))[['Артикул', 'Наценка']]\
             .rename(columns={'Наценка': 'Наценка с учетом скидок'}),
             on='Артикул',
             how='left')
else:
    df['Со скидками за ваш счёт, ₽'] = None
    df['Цена с учетом скидок'] = df['Цена']
    df['Наценка с учетом скидок'] = df['Наценка']
    
df = df.merge(price_for_PRICES[['Артикул', 
                                'Софинансирование скидок', 
                                'Unnamed: 9', 
                                'Порог для привлекательной цены, ₽',
                                'Порог для умеренно привлекательной цены, ₽']]\
              .rename(columns={'Софинансирование скидок': 'Установленный вами порог, ₽',
                               'Unnamed: 9': 'Оптимальный порог по расчётам Маркета, ₽'}), 
                      on='Артикул', 
                      how='left')

df = df.merge(price_for_PRICES[['Артикул', 'Ваша цена, ₽']]\
              .rename(columns={'Ваша цена, ₽': 'Ваша цена, ₽ FBY'}), on='Артикул', how='left')

df = df.merge(price_for_PRICES_himki[['Артикул', 'Ваша цена, ₽']]\
              .rename(columns={'Ваша цена, ₽': 'Ваша цена, ₽ FBS Х'}), on='Артикул', how='left')

df = df.merge(price_for_PRICES_irkutsk[['Артикул', 'Ваша цена, ₽']]\
              .rename(columns={'Ваша цена, ₽': 'Ваша цена, ₽ FBS И'}), on='Артикул', how='left')

In [221]:
ll = list(df.columns)

In [222]:
if len(path_AC_future) > 0:
    for AC in path_AC_future:   
        ya_AC = pd.read_excel(path_downloads + AC, sheet_name='Товары и цены')
        name_AC = ','.join(ya_AC.columns[0].split(':')[1:])
        ya_AC = ya_AC.iloc[4:, :].reset_index(drop=True)
        ya_AC.columns = list(ya_AC.loc[0].values)
        if str(list(set(ya_AC.iloc[0, :]))[0]) == 'nan':
            ya_AC = pd.read_excel(path_downloads + AC, sheet_name='Товары и цены')
            name_AC = ','.join(ya_AC.columns[0].split(':')[1:])
            ya_AC = ya_AC.iloc[5:, :].reset_index(drop=True)
            ya_AC.columns = list(ya_AC.loc[0].values)
        try:
            ya_AC = ya_AC[['Ваш SKU', 'Максимальная цена для участия в акции']]
        except:
            ya_AC = ya_AC[['Ваш SKU', 'Максимальный порог цены для участия в акции']]
        ya_AC.columns = ['Артикул', 'Цена']
        Q = []
        for i in range(len(ya_AC)):
            try:
                int(ya_AC.loc[i]['Цена'])
            except:
                Q.append(i)
        ya_AC = ya_AC.query('index not in @Q').reset_index(drop=True)
        ya_AC['Артикул'] = ya_AC['Артикул'].astype(str)
        ya_AC['Цена'] = ya_AC['Цена'].astype(int)
        if len(ya_AC) - len(list(set(ya_AC['Артикул']))) != 0:
            exit()
        else:
            df = df.merge(calculated_features(yandex_orders_metrics_AC.merge(ya_AC, on='Артикул', how='inner'))[['Артикул', 'Цена', 'Наценка']]\
                          .rename(columns={'Цена': 'Цена' + f'_{name_AC}',
                                           'Наценка': 'Наценка' + f'_{name_AC}'}), 
                          on='Артикул', 
                          how='left')
else:
    None

In [223]:
df_columns_2 = list(df.columns)

In [224]:
ac_columns = [x for x in list(df_columns_2) if x not in list(df_columns_1)]

In [225]:
pd.DataFrame(ac_columns + list(set(ll) - set(rr)), 
             columns=['Col_for_not_del']).to_excel(path_downloads + 'Col_for_not_del_ya.xlsx', index=False)

In [226]:
# df = df.merge(yandex_orders_metrics_AC, on='Артикул', how='left')

In [227]:
if len(yandex_orders_metrics_11) - len(list(set(yandex_orders_metrics_11['Артикул']))) == 0:
    df = df.merge(yandex_orders_metrics_11[['Артикул', 'размещение на витрине %']], on='Артикул', how='left')
else:
    print('Ошибка')
    int('error')

In [228]:
df['размещение на витрине %'].loc[df[df['Артикул']=='Итого'].index[0]] = f"mean %: {round(df['размещение на витрине %'][:-1].mean(), 2)}"

In [229]:
if len(stocks_ya_metrics) - len(list(set(stocks_ya_metrics['Артикул']))) == 0:

    df_dop_stocks_features = stocks_ya_metrics[['Артикул']+dt_col(stocks_ya_metrics)[-2:]].copy()

    df_dop_stocks_features.loc[len(df_dop_stocks_features)] = ['Итого']\
    + [df_dop_stocks_features[list(df_dop_stocks_features)[1]].sum()]\
    + [df_dop_stocks_features[list(df_dop_stocks_features)[2]].sum()]

    df_dop_stocks_features['diff_stocks'] = df_dop_stocks_features[list(df_dop_stocks_features)[-1]]\
    - df_dop_stocks_features[list(df_dop_stocks_features)[-2]]

    df_dop_stocks_features = df_dop_stocks_features[['Артикул', 'diff_stocks']]
    
    df = df.merge(df_dop_stocks_features, on='Артикул', how='left')
else:
    print('Ошибка')
    int('error')

In [230]:
df

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,"Оптимальный порог по расчётам Маркета, ₽","Порог для привлекательной цены, ₽","Порог для умеренно привлекательной цены, ₽","Ваша цена, ₽ FBY","Ваша цена, ₽ FBS Х","Ваша цена, ₽ FBS И","Цена_ Флеш, космические товары. 11.04–12.04","Наценка_ Флеш, космические товары. 11.04–12.04",размещение на витрине %,diff_stocks
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,101976154101,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,649.0,649.0,649.0,NaN,NaN,4.0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945750,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,649.0,649.0,649.0,NaN,NaN,4.0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,101312041369,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,722.0,722.0,722.0,NaN,NaN,4.0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945752,архив,0,0,0,0,0,0,...,NaN,NaN,NaN,601.0,601.0,601.0,NaN,NaN,4.0,0
4,16920,Нож канцелярский металлический,101827068308,текущий,0,0,0,0,0,0,...,170,NaN,NaN,184.0,184.0,184.0,NaN,NaN,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,Z0401,Ваза для цветов и сухоцветов решетка,101986518829,текущий Дарья,0,0,0,3,1,1,...,1270,1421.0,1440.0,1380.0,1380.0,1380.0,NaN,NaN,4.0,-2
1152,Z0402,Ваза 5 колб,102591438907,архив,0,0,0,0,0,0,...,NaN,NaN,NaN,3900.0,3900.0,3900.0,NaN,NaN,4.0,0
1153,Z0500,Декантер с бокалами,101994522856,текущий Дарья,0,0,0,0,1,0,...,3247,NaN,NaN,3510.0,3510.0,3510.0,NaN,NaN,4.0,0
1154,Z0501,Графин для воды,102071515515,текущий Дарья,0,1,0,0,0,0,...,1320,NaN,NaN,1372.0,1372.0,1372.0,NaN,NaN,4.0,0


## Цена до скидки

In [231]:
# dop_price = pd.read_excel(path_price,
#                           sheet_name='Список товаров', 
#                           skiprows=2,
#                           usecols=[2, 5])

# dop_price.columns = ['Артикул', 'Цена до скидки']
# dop_price['Артикул'] = dop_price['Артикул'].astype(str)
# dop_price['Цена до скидки'] = dop_price['Цена до скидки'].astype(float)


df.insert(list(df.columns).index('Цена')+1, 
                  'Цена до скидки', 
                  list(df[['Артикул']]\
                       .merge(dop_price, 
                              on='Артикул', 
                              how='left')['Цена до скидки']))

In [232]:
df

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,"Оптимальный порог по расчётам Маркета, ₽","Порог для привлекательной цены, ₽","Порог для умеренно привлекательной цены, ₽","Ваша цена, ₽ FBY","Ваша цена, ₽ FBS Х","Ваша цена, ₽ FBS И","Цена_ Флеш, космические товары. 11.04–12.04","Наценка_ Флеш, космические товары. 11.04–12.04",размещение на витрине %,diff_stocks
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,101976154101,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,649.0,649.0,649.0,NaN,NaN,4.0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945750,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,649.0,649.0,649.0,NaN,NaN,4.0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,101312041369,отказ,0,0,0,0,0,0,...,NaN,NaN,NaN,722.0,722.0,722.0,NaN,NaN,4.0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,102080945752,архив,0,0,0,0,0,0,...,NaN,NaN,NaN,601.0,601.0,601.0,NaN,NaN,4.0,0
4,16920,Нож канцелярский металлический,101827068308,текущий,0,0,0,0,0,0,...,170,NaN,NaN,184.0,184.0,184.0,NaN,NaN,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,Z0401,Ваза для цветов и сухоцветов решетка,101986518829,текущий Дарья,0,0,0,3,1,1,...,1270,1421.0,1440.0,1380.0,1380.0,1380.0,NaN,NaN,4.0,-2
1152,Z0402,Ваза 5 колб,102591438907,архив,0,0,0,0,0,0,...,NaN,NaN,NaN,3900.0,3900.0,3900.0,NaN,NaN,4.0,0
1153,Z0500,Декантер с бокалами,101994522856,текущий Дарья,0,0,0,0,1,0,...,3247,NaN,NaN,3510.0,3510.0,3510.0,NaN,NaN,4.0,0
1154,Z0501,Графин для воды,102071515515,текущий Дарья,0,1,0,0,0,0,...,1320,NaN,NaN,1372.0,1372.0,1372.0,NaN,NaN,4.0,0


## прибыль с единицы товара

In [233]:
df = df.merge(yandex_orders_metrics_11[['Артикул', 
                                        'Прибыль']]\
              .rename(columns={'Прибыль': 'прибыль с единицы товара'}),
              on='Артикул', 
              how='left') 
df['прибыль с единицы товара'] = df['прибыль с единицы товара'].round(2)

## Создание таблицы с изменением цен

In [234]:
if len(ya_price_metrics) - len(list(set(ya_price_metrics['Артикул']))) !=0:
    print('error')
    int('error')
else:
    ya_price_difference = ya_price_metrics[['Артикул']+dt_col(ya_price_metrics)[-6:]].copy()
    cols_for_analysis = list(ya_price_difference.columns)[1:][::-1]
    for i in range(len(cols_for_analysis)-1):
        ya_price_difference['Изм. цены ' + pd.to_datetime(cols_for_analysis[i])\
                            .strftime('%d.%m')] = ya_price_difference[cols_for_analysis[i]]\
                                                    - ya_price_difference[cols_for_analysis[i+1]]
#         ozon_price_difference['Цена '+ pd.to_datetime(cols_for_analysis[i])\
#                             .strftime('%d.%m')] = ozon_price_difference[cols_for_analysis[i]]
    ya_price_difference = ya_price_difference.drop(cols_for_analysis, axis=1)
    ya_price_difference = ya_price_difference[['Артикул']\
                                             + list(ya_price_difference.columns[1:])[::-1]]
    try:
        C_price = pd.read_excel(path_downloads + 'Целевая цена.xlsx',
                                sheet_name='yandex', 
                                dtype={'Артикул': 'str'})
        ya_price_difference = ya_price_difference.merge(C_price, on='Артикул', how='left')
    except:
        None
    ya_price_difference.to_excel(path_downloads + 'Изменения цены яндекс.xlsx', index=False)

## Сохранение наценки

In [235]:
ya_markup_metrics = pd.read_excel(wwm.path_ya_markup_metrics,
                                  dtype={'Артикул': 'str'})
if len(ya_markup_metrics) - len(list(set(ya_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в ya_markup_metrics')
    exit()
ya_markup_new = yandex_orders_metrics_11[['Артикул', 'Наценка']]\
                .rename(columns={'Наценка': const.dt_now.strftime('%d-%B-%Y')}).copy()
if len(ya_markup_new) - len(list(set(ya_markup_new['Артикул']))) != 0:
    print('Неуникальные артикулы в ya_markup_new')
    exit()
col = wwm.dt_col(ya_markup_new)[0]
for i in range(len(ya_markup_new)):
    art = ya_markup_new.loc[i]['Артикул']
    if art in list(set(ya_price_metrics['Артикул'])):
        if ya_price_metrics[ya_price_metrics['Артикул']==art][ya_price_metrics.columns[-1]].values[0] == 0:
            ya_markup_new.loc[i, col] = None
if col in ya_markup_metrics.columns:
    ya_markup_metrics = ya_markup_metrics.drop(col, axis=1)
    ya_markup_metrics = ya_markup_metrics.merge(ya_markup_new, on='Артикул', how='outer')
else:
    ya_markup_metrics = ya_markup_metrics.merge(ya_markup_new, on='Артикул', how='outer')
if len(ya_markup_metrics) - len(list(set(ya_markup_metrics['Артикул']))) != 0:
    print('Неуникальные артикулы в ya_markup_metrics')
    exit()
ya_markup_metrics.to_excel(wwm.path_ya_markup_metrics, index=False)

In [236]:
df = wwm.rename_data_from_reestr(df, 
                                 'Название', 
                                 reestr=reestr)

yandex_orders_metrics_11 = wwm.rename_data_from_reestr(yandex_orders_metrics_11, 
                                                       'Название', 
                                                       reestr=reestr)

## Выгрузка

In [237]:
df.to_excel(path_algoritm + 'YANDEX_ORDERS/yandex_orders_metrics_итоговые_предобработанные.xlsx', index=False)


In [238]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_YA_df.xlsx") as writer:


        yandex_orders_metrics_22.to_excel(writer, sheet_name="ya_orders_metrics", index=False)
        for column in yandex_orders_metrics_22:
            column_width = max(yandex_orders_metrics_22[column].astype(str).map(len).max(), len(column))
            col_idx = yandex_orders_metrics_22.columns.get_loc(column)
            writer.sheets['ya_orders_metrics'].set_column(col_idx, col_idx, column_width)



        ya_price_metrics.drop_duplicates().to_excel(writer, sheet_name="ya_price_metrics", index=False)
        for column in ya_price_metrics:
            column_width = max(ya_price_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = ya_price_metrics.columns.get_loc(column)
            writer.sheets['ya_price_metrics'].set_column(col_idx, col_idx, column_width)



        stocks_ya_metrics.drop_duplicates().to_excel(writer, sheet_name="stocks_ya_metrics", index=False)
        for column in stocks_ya_metrics:
            column_width = max(stocks_ya_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_ya_metrics.columns.get_loc(column)
            writer.sheets['stocks_ya_metrics'].set_column(col_idx, col_idx, column_width)
            
            
            
        stocks_fbs_ya_metrics.to_excel(writer, sheet_name="stocks_fbs_ya_metrics", index=False)
        for column in stocks_fbs_ya_metrics:
            column_width = max(stocks_fbs_ya_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_fbs_ya_metrics.columns.get_loc(column)
            writer.sheets['stocks_fbs_ya_metrics'].set_column(col_idx, col_idx, column_width)







        yandex_orders_metrics_11.to_excel(writer, sheet_name="ya_full_features_metrics", index=False)
        for column in yandex_orders_metrics_11:
            column_width = max(yandex_orders_metrics_11[column].astype(str).map(len).max() + 3, len(column))
            col_idx = yandex_orders_metrics_11.columns.get_loc(column)
            writer.sheets['ya_full_features_metrics'].set_column(col_idx, col_idx, column_width)



        df.to_excel(writer, sheet_name="ya_full_df", index=False)
        for column in df:
            column_width = max(df[column].astype(str).map(len).max() + 3, len(column))
            col_idx = df.columns.get_loc(column)
            writer.sheets['ya_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()

In [239]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [240]:
data = df

data['Артикул'] = data['Артикул'].astype(str)

print(len(list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))))
print(len(data)-len(list(set(data['Артикул']))))

1
0


In [241]:
data[data['Артикул'].isin(data.loc[data['Артикул'].duplicated(), 'Артикул'].tolist())]

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,"Порог для привлекательной цены, ₽","Порог для умеренно привлекательной цены, ₽","Ваша цена, ₽ FBY","Ваша цена, ₽ FBS Х","Ваша цена, ₽ FBS И","Цена_ Флеш, космические товары. 11.04–12.04","Наценка_ Флеш, космические товары. 11.04–12.04",размещение на витрине %,diff_stocks,прибыль с единицы товара


In [242]:
df[df['Артикул']=='29314']

,Артикул,Название,Номенклатура,Статус,21-February-2024,22-February-2024,23-February-2024,24-February-2024,25-February-2024,26-February-2024,...,"Порог для привлекательной цены, ₽","Порог для умеренно привлекательной цены, ₽","Ваша цена, ₽ FBY","Ваша цена, ₽ FBS Х","Ваша цена, ₽ FBS И","Цена_ Флеш, космические товары. 11.04–12.04","Наценка_ Флеш, космические товары. 11.04–12.04",размещение на витрине %,diff_stocks,прибыль с единицы товара
226,29314,Набор бигуди-липучек 6см 6 шт.,102075338857,текущий Дарья,2,1,5,1,5,2,...,NaN,NaN,277.0,277.0,277.0,NaN,NaN,4.0,0,86.8


In [243]:
df['Наценка'].isna().sum()

24

In [244]:
list(df.columns)

['Артикул',
 'Название',
 'Номенклатура',
 'Статус',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-April-2024',
 '04-April-2024',
 '05-April-2024

In [245]:
exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
